In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="7" 

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from configs.CC import Config
from peleenet import build_net

from utils.core import anchors, init_net, get_dataloader,\
print_info, image_forward, nms_process
from utils.timer import Timer

from layers.functions import Detect, PriorBox
from data import BaseTransform

from tqdm import tqdm
import pickle

In [3]:
def test_net(save_folder, net, detector, cuda, testset, transform, max_per_image=300, thresh=0.005):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    num_images = len(testset)
    print_info('=> Total {} images to test.'.format(
        num_images), ['yellow', 'bold'])
    all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]

    _t = {'im_detect': Timer(), 'misc': Timer()}
    det_file = os.path.join(save_folder, 'detections.pkl')
    tot_detect_time, tot_nms_time = 0, 0
    print_info('Begin to evaluate', ['yellow', 'bold'])
    for i in tqdm(range(num_images)):
        img = testset.pull_image(i)
        # step1: CNN detection
        _t['im_detect'].tic()
        boxes, scores = image_forward(
            img, net, cuda, priors, detector, transform)
        detect_time = _t['im_detect'].toc()
        # step2: Post-process: NMS
        _t['misc'].tic()
        nms_process(num_classes, i, scores, boxes, cfg,
                    thresh, all_boxes, max_per_image)
        nms_time = _t['misc'].toc()

        tot_detect_time += detect_time if i > 0 else 0
        tot_nms_time += nms_time if i > 0 else 0

    with open(det_file, 'wb') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)
    print_info('===> Evaluating detections', ['yellow', 'bold'])
    testset.evaluate_detections(all_boxes, save_folder)
    print_info('Detect time per image: {:.3f}s'.format(
        tot_detect_time / (num_images - 1)))
    print_info('Nms time per image: {:.3f}s'.format(
        tot_nms_time / (num_images - 1)))
    print_info('Total time per image: {:.3f}s'.format(
        (tot_detect_time + tot_nms_time) / (num_images - 1)))
    print_info('FPS: {:.3f} fps'.format(
        (num_images - 1) / (tot_detect_time + tot_nms_time)))

## Build net

In [4]:
cfgfile = "./configs/Pelee_VOC.py"

cfg = Config.fromfile(cfgfile)
net = build_net('test', cfg.model.input_size, cfg.model)

path2pretrained = "/workspace/raid/data/jgusak/Pelee.Pytorch/Pelee_VOC.pth"
init_net(net, cfg, path2pretrained)

net.eval()

Loading resume network...


PeleeNet(
  (features): Sequential(
    (stemblock): _StemBlock(
      (stem1): conv_bn_relu(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2a): conv_bn_relu(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2b): conv_bn_relu(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem3): conv_bn_relu(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0

## Generate priorboxes

In [6]:
anchor_config = anchors(cfg.model)
priorbox = PriorBox(anchor_config)

priorbox = PriorBox(anchor_config)
with torch.no_grad():
    priors = priorbox.forward()
    if cfg.test_cfg.cuda:
        priors = priors.cuda()
        
num_classes = cfg.model.num_classes

## Create testset


In [8]:
dataset_name = 'VOC'

_set = 'eval_sets' # or '_test_sets'
testset = get_dataloader(cfg, dataset_name, _set)

len(testset)

4952

## Put model to the device


In [9]:
if cfg.test_cfg.cuda:
    net = net.cuda()
    cudnn.benckmark = True
else:
    net = net.cpu()

## Create detector


In [10]:
detector = Detect(num_classes, cfg.loss.bkg_label, anchor_config)
type(detector)

layers.functions.detection.Detect

## Save folder, preprocessor

In [11]:
save_folder = os.path.join(cfg.test_cfg.save_folder, dataset_name)
print(save_folder)

_preprocess = BaseTransform(
        cfg.model.input_size, cfg.model.rgb_means, (2, 0, 1))

/workspace/raid/data/jgusak/Pelee.Pytorch/eval/VOC


## Test

In [ ]:
test_net(save_folder,
         net,
         detector,
         cfg.test_cfg.cuda,
         testset,
         transform=_preprocess,
         max_per_image=cfg.test_cfg.topk,
         thresh=cfg.test_cfg.score_threshold)

dict(net.features.named_children()).keys()

## Draft

In [23]:
from torchvision.models import resnet18, resnet50

In [26]:
mm=resnet18()

In [27]:
dict(mm.named_children()).keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [28]:
x= torch.randn(1, 3, 224, 224)

In [38]:
x= torch.randn(1, 3, 224, 224)

for l in net.features:
    x = l(x)
    print(l._get_name(), x.shape)
    
print('\n')
x= torch.randn(1, 3, 224, 224)
x = mm.maxpool(mm.conv1(x))

for l in [mm.layer1, mm.layer2, mm.layer3, mm.layer4]:
    x = l(x)
    print(l._get_name(), x.shape)

_StemBlock torch.Size([1, 32, 56, 56])
_DenseBlock torch.Size([1, 128, 56, 56])
conv_bn_relu torch.Size([1, 128, 56, 56])
AvgPool2d torch.Size([1, 128, 28, 28])
_DenseBlock torch.Size([1, 256, 28, 28])
conv_bn_relu torch.Size([1, 256, 28, 28])
AvgPool2d torch.Size([1, 256, 14, 14])
_DenseBlock torch.Size([1, 512, 14, 14])
conv_bn_relu torch.Size([1, 512, 14, 14])
AvgPool2d torch.Size([1, 512, 7, 7])
_DenseBlock torch.Size([1, 704, 7, 7])
conv_bn_relu torch.Size([1, 704, 7, 7])


Sequential torch.Size([1, 64, 56, 56])
Sequential torch.Size([1, 128, 28, 28])
Sequential torch.Size([1, 256, 14, 14])
Sequential torch.Size([1, 512, 7, 7])


In [41]:
net.extras

ModuleList(
  (0): conv_bn_relu(
    (conv): Conv2d(704, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): conv_bn_relu(
    (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): conv_bn_relu(
    (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): conv_bn_relu(
    (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (4): conv_bn_relu(
    (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  

In [42]:
net.features

Sequential(
  (stemblock): _StemBlock(
    (stem1): conv_bn_relu(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem2a): conv_bn_relu(
      (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem2b): conv_bn_relu(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem3): conv_bn_relu(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (denseblock1): _DenseBlo